#### _Matin's Lab Rotation 3 - BCFO_ | [NEXT](03_community_dFBA_growth_objective) | [PREVIOUS](01-0_simulation_xylitol_strain.ipynb)
# 03. Simulation of the Xylitol Producing Strain
In this section, we would like to see if the model can:
* Uptake Carbon: Glucose and Xylose
* Grow: Biomass & ATP
* Produce: Acetate & Xylitol

## Load Library

In [1]:
from cameo import load_model
from cameo import fba
from escher import Builder
import pandas as pd
import cobra

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
def simulate_model(experiment_id, model, objective, target, metabolite):
    model.objective = objective
    solution = fba(model)
    f_data = solution.fluxes
    f_data = f_data.rename(experiment_id)
    solution_result = []
    for i in target:
        flux = solution.fluxes.get(i)
        solution_result.append(flux)
    data = [experiment_id] + solution_result
    #print(data)
    df_target = pd.DataFrame(columns=[["Model"]+target], data=[data])
    print("experiment:", experiment_id, "| evaluate compound:", model.metabolites.get_by_id(metabolite).summary())
    return df_target, f_data

def run_series_simulation(experiment, model, mutant, target, metabolite):
    solution_fluxes = pd.DataFrame()
    df = pd.DataFrame()
    for i in experiment.index:
        experiment_id = experiment.loc[i, "experiment_id"]
        if experiment.loc[i, "model_id"] != "model":
            m = mutant
        else:
            m = model
        objective = experiment.loc[i, "objective"]
        data, sol = simulate_model(experiment_id, m, objective, target, metabolite)
        solution_fluxes = solution_fluxes.append(sol)
        df = df.append(data)
        print("****************************************", "\n")
    return df, solution_fluxes

def visualize_flux(model, objective, map_data='map/iJO1366.Central metabolism_rotation3.json'):
    model.objective = objective
    flux = fba(model).fluxes
    builder = Builder(
        map_json = map_data,
        reaction_data = flux
    )
    return builder

## Load Model

In [3]:
# WT for comparison
model1 = load_model('iML1515')
model1.solver = 'gurobi'
model1

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f6d11840b50
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [4]:
# load mutant
mutant = load_model('result/iML1515_isobutyric_strain.xml')
mutant.solver = 'gurobi'
mutant

Name,iML1515_iso_obj
Memory address,0x07f6d124841d0
Number of metabolites,1881
Number of reactions,2716
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


## Define Media: Glucose & Xylose

In [16]:
# define media composition
media_components = [["EX_xyl__D_e", 75],
                    ["EX_glc__D_e", 75],
                    ["EX_ac_e", 75],
                    ["EX_o2_e", 1000]
                   ]

# change the media in the model
medium = model1.medium
for i in media_components:
    medium[i[0]] = i[1]
    model1.medium = medium
#print(model.medium)

# change the media in the mutant
medium = mutant.medium
for i in media_components:
    medium[i[0]] = i[1]
    mutant.medium = medium
#print(mutant.medium)

# WT Grow on Glu & Xyl

## WT Biomass Objective

In [17]:
visualize_flux(model1, "BIOMASS_Ec_iML1515_core_75p37M")

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0, 'SHK3Dr': 5.056570408279506, 'N…

## WT under Anaerobic Condition

In [18]:
media_components = [["EX_o2_e", 0]]
medium = model1.medium
for i in media_components:
    medium[i[0]] = i[1]
    model1.medium = medium
visualize_flux(model1, "BIOMASS_Ec_iML1515_core_75p37M")

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0, 'SHK3Dr': 1.053008630690547, 'N…

# Mutant Growth on Glu & Xyl
## Mt Growth Objective

In [19]:
visualize_flux(mutant, "BIOMASS_Ec_iML1515_core_75p37M")

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.2317559358800193, 'SHK3Dr': 3.2692…

## Mt Xylitol Objective

In [29]:
reaction_1 = mutant.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M')
#reaction_1 = mutant.reactions.get_by_id('ATPM')
reaction_2 = mutant.reactions.get_by_id('EX_isobuta_e')

new_objective = {reaction_1: 0.93, reaction_2:0.07}
visualize_flux(mutant, new_objective)

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0, 'SHK3Dr': 1.972716780363308, 'N…

In [21]:
mutant.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,75,2,25.00%
ca2_e,EX_ca2_e,0.04465,0,0.00%
cl_e,EX_cl_e,0.04465,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0002145,0,0.00%
cu2_e,EX_cu2_e,0.006082,0,0.00%
fe2_e,EX_fe2_e,0.1378,0,0.00%
glc__D_e,EX_glc__D_e,75,6,75.00%
k_e,EX_k_e,1.674,0,0.00%
mg2_e,EX_mg2_e,0.07442,0,0.00%
mn2_e,EX_mn2_e,0.005928,0,0.00%


In [22]:
mutant.id = mutant.id+"_xy_strain_crispr_off"
cobra.io.write_sbml_model(mutant, "result/model_for_consortia/iML1515_xylitol_strain_crispr_off.xml")
cobra.io.save_json_model(mutant, "result/model_for_consortia/iML1515_xylitol_strain_crispr_off.json")
mutant.metabolites.ac_c.summary()

Percent,Flux,Reaction,Definition
3.17%,2.537,ACODA,acorn_c + h2o_c --> ac_c + orn_c
93.70%,74.96,ACt2rpp,ac_p + h_p <=> ac_c + h_c
0.06%,0.04465,ACt4pp,ac_p + na1_p --> ac_c + na1_c
2.65%,2.123,CYSS,acser_c + h2s_c --> ac_c + cys__L_c + h_c
0.42%,0.3338,UHGADA,h2o_c + u3aga_c --> ac_c + u3hga_c
Percent,Flux,Reaction,Definition
100.00%,-79.99,ACKr,ac_c + atp_c <=> actp_c + adp_c


In [23]:
visualize_flux(crispri(mutant, "on"), new_objective)

NameError: name 'crispri' is not defined

In [ ]:
mutant.summary()

In [ ]:
mutant.id = mutant.id+"_xy_strain_crispr_on"
cobra.io.write_sbml_model(mutant, "result/model_for_consortia/iML1515_xylitol_strain_crispr_on.xml")
cobra.io.save_json_model(mutant, "result/model_for_consortia/iML1515_xylitol_strain_crispr_on.json")
mutant.metabolites.ac_c.summary()